# Glucose

This notebook is to be used to examine blood glucose record entries in the Apple Health export.

In [ ]:
from datetime import datetime, timedelta, timezone
import os

import seaborn as sns

sns.set_theme(style="darkgrid")

In [ ]:
from apple_health_analysis.healthdata import get_healthdata

print("Getting HealthData -- this may take a while...")
healthdata = get_healthdata(use_cache=True)
print("Done getting HealthData")

In [ ]:
glucose_records = healthdata.records[
    healthdata.records.record_type == "HKQuantityTypeIdentifierBloodGlucose"
]
print(
    f"{len(glucose_records)} glucose records found out of {len(healthdata.records)} records total"
)

In [ ]:
print(glucose_records.unit.describe())

In [ ]:
print(glucose_records.value.describe())

## Calibration

Some CGMs may need [manual calibration](https://www.nutrisense.io/blog/how-cgms-work-and-accuracy?srsltid=AfmBOoo09RrYXxYjmHb-QHoGq0A6G-akYhGmxFZmdU41zge3S178ACFp) based on blood glucometer readings (i.e., finger sticks). Before performing further analysis, one may want to first calibrate their readings.

In [ ]:
sns.lineplot(
    x="start_date",
    y="value",
    data=glucose_records,
)

In [ ]:
glucose_records_cal = glucose_records.copy(deep=True)

# Put whatever calibration makes sense for you here

# For instance, the two [Stelos](https://www.stelo.com/oura-ring) I tried with Oura needed calibration.
# The first unit might have been defective generally (40 mg/dL above glucometer), but the second was more reasonable (only ~10-15 mg/dL above).
switch_datetime = datetime(2025, 6, 15, hour=16, tzinfo=timezone.utc)  # Approximately
glucose_records_cal.loc[glucose_records_cal.start_date <= switch_datetime, "value"] -= (
    40.0
)
glucose_records_cal.loc[glucose_records_cal.start_date > switch_datetime, "value"] -= (
    15.0
)

In [ ]:
sns.lineplot(
    x="start_date",
    y="value",
    data=glucose_records_cal,
)

## glucotype.stanford.edu

Researchers at Stanford University created a [web app](https://adaychen.shinyapps.io/shinyspecclust/) to understand one's "glucotype", a phenotype of how one regulates their blood glucose. It takes a simple TSV file as input, which we can create here from the calibrated readings.

In [ ]:
from apple_health_analysis.healthdata import APPLE_HEALTH_DIR

local_timezone = timezone(timedelta(hours=-4))  # EDT

with open(os.path.join(APPLE_HEALTH_DIR, "glucotype.tsv"), "w") as fd:
    fd.write("GlucoseDisplayTime\tGlucoseValue\n")
    for row_tuple in glucose_records_cal.itertuples():
        dt_str = row_tuple.start_date.replace(tzinfo=local_timezone).strftime(
            "%Y-%m-%d %H:%M:%S"
        )
        fd.write(f"{dt_str}\t{int(row_tuple.value)}\n")